# Tutorial: Entity Extraction with DSPy

This tutorial demonstrates how to perform entity extraction using the CoNLL 2003 dataset with DSPy. The focus is on extracting entities referring to people. We will:

- Extract and label entities from the CoNLL 2003 dataset that refer to people
- Define a DSPy program for extracting entities that refer to people
- Optimize and evaluate the program on a subset of the CoNLL 2003 dataset

By the end of this tutorial, you'll understand how to structure tasks in DSPy using signatures and modules, evaluate your system's performance, and improve its quality with optimizers.

Install the latest version of DSPy and follow along. If you're looking instead for a conceptual overview of DSPy, this [recent lecture](https://www.youtube.com/live/JEMYuzrKLUw) is a good place to start.

In [1]:
# Install the latest version of DSPy
%pip install -U dspy-ai>=2.6.0
# Install the Hugging Face datasets library to load the CoNLL 2003 dataset
%pip install datasets

## Load and Prepare the Dataset

In this section, we prepare the CoNLL 2003 dataset, which is commonly used for entity extraction tasks. The dataset includes tokens annotated with entity labels such as persons, organizations, and locations.

We will:
1. Load the dataset using the Hugging Face `datasets` library.
2. Define a function to extract tokens referring to people.
3. Slice the dataset to create smaller subsets for training and testing.

DSPy expects examples in a structured format, so we'll also transform the dataset into DSPy `Examples` for easy integration.

In [2]:
import os
import tempfile
from datasets import load_dataset
from typing import Dict, Any, List
import dspy

def load_conll_dataset() -> dict:
    """
    Loads the CoNLL 2003 dataset into train, validation, and test splits.
    
    Returns:
        dict: Dataset splits with keys 'train', 'validation', and 'test'.
    """
    with tempfile.TemporaryDirectory() as temp_dir:
        # Use a temporary Hugging Face cache directory for compatibility with certain hosted notebook
        # environments that don't support the default Hugging Face cache directory
        os.environ["HF_DATASETS_CACHE"] = temp_dir
        return load_dataset("conll2003", trust_remote_code=True)

def extract_people_entities(data_row: Dict[str, Any]) -> List[str]:
    """
    Extracts entities referring to people from a row of the CoNLL 2003 dataset.
    
    Args:
        data_row (Dict[str, Any]): A row from the dataset containing tokens and NER tags.
    
    Returns:
        List[str]: List of tokens tagged as people.
    """
    return [
        token
        for token, ner_tag in zip(data_row["tokens"], data_row["ner_tags"])
        if ner_tag in (1, 2)  # CoNLL entity codes 1 and 2 refer to people
    ]

def prepare_dataset(data_split, start: int, end: int) -> List[dspy.Example]:
    """
    Prepares a sliced dataset split for use with DSPy.
    
    Args:
        data_split: The dataset split (e.g., train or test).
        start (int): Starting index of the slice.
        end (int): Ending index of the slice.
    
    Returns:
        List[dspy.Example]: List of DSPy Examples with tokens and expected labels.
    """
    return [
        dspy.Example(
            tokens=row["tokens"],
            expected_extracted_people=extract_people_entities(row)
        ).with_inputs("tokens")
        for row in data_split.select(range(start, end))
    ]

# Load the dataset
dataset = load_conll_dataset()

# Prepare the training and test sets
train_set = prepare_dataset(dataset["train"], 0, 50)
test_set = prepare_dataset(dataset["test"], 10, 60)

## Set Up DSPy and create an Entity Extraction Program

Here, we define a DSPy program for extracting entities referring to people from tokenized text.

Then, we configure DSPy to use a particular language model (`gpt-4o-mini`) for all invocations of the program.

**Key DSPy Concepts Introduced:**
- **Signatures:** Define structured input/output schemas for your program.
- **Modules:** Encapsulate program logic in reusable, composable units.

Specifically, we'll:
- Create a `PeopleExtractionSignature` to specify the input (`tokens`) and output (`extracted_people`, `rationale`) fields.
- Define a `PeopleExtractor` module that predicts the extracted people based on the input tokens using language model (LM) prompting.
- Use the `dspy.LM` class and `dspy.settings.configure()` method to configure the language model that DSPy will use when invoking the program.

In [3]:
from typing import List

class PeopleExtractionSignature(dspy.Signature):
    """
    Extract tokens referring to specific people from a list of string tokens.

    The tokens may be intentionally designed to be difficult. Ensure that each extracted entity is actually a person (e.g., not a location or the name of an organization).
    Ensure that the extracted entity appears in the original list of tokens. Do not combine multiple tokens into a single entity.
    """
    tokens: List[str] = dspy.InputField(desc="Tokenized text, which may or may not contain tokens referring to people")
    extracted_people: List[str] = dspy.OutputField(desc="A list of all tokens referring to specific people extracted from the tokenized text")
    rationale: str = dspy.OutputField(desc="A detailed rationale for why each token referring to a person / people was extracted and why no additional tokens were extracted")

class PeopleExtractor(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract_people = dspy.Predict(PeopleExtractionSignature)

    def forward(self, tokens: List[str]) -> List[str]:
        return self.extract_people(tokens=tokens).extracted_people

# Initialize the entity extractor program
extractor_program = PeopleExtractor()

Here, we tell DSPy to use OpenAI's `gpt-4o-mini` model in our program. To authenticate, DSPy reads your `OPENAI_API_KEY`. You can easily swap this out for [other providers or local models](https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb).

In [4]:
lm = dspy.LM(model="openai/gpt-4o-mini")
dspy.settings.configure(lm=lm)

## Define Metric and Evaluation Functions

In DSPy, evaluating a program's performance is critical for iterative development. A good evaluation framework allows us to:
- Measure the quality of our program's outputs.
- Compare outputs against ground-truth labels.
- Identify areas for improvement.

**What We'll Do:**
- Define a custom metric (`extraction_correctness_metric`) to evaluate whether the extracted entities match the ground truth.
- Create an evaluation function (`evaluate_correctness`) to apply this metric to a training or test dataset and compute the overall accuracy.

The evaluation function uses DSPy's `Evaluate` utility to handle parallelism and visualization of results.

In [5]:
def extraction_correctness_metric(example: dspy.Example, predicted_extracted_people: List[str], trace=None) -> bool:
    """
    Computes correctness of entity extraction predictions.
    
    Args:
        example (dspy.Example): The dataset example containing expected people entities.
        predicted_extracted_people (List[str]): Predicted list of extracted people entities.
        trace: Optional trace object for debugging.
    
    Returns:
        bool: True if predictions match expectations, False otherwise.
    """
    return predicted_extracted_people == example.get("expected_extracted_people")

def evaluate_correctness(program, evaluation_set):
    """
    Evaluates a DSPy program using a defined test set and metric.
    
    Args:
        program: The program to evaluate.
        evaluation_set: The evaluation dataset.
    """
    evaluation = dspy.Evaluate(
        devset=evaluation_set,
        metric=extraction_correctness_metric,
        num_threads=24,
        display_progress=True,
        display_table=True
    )
    evaluation(program)

## Evaluate Initial Extractor

Before optimizing our program, we need a baseline evaluation to understand its current performance. This helps us:
- Establish a reference point for comparison after optimization.
- Identify potential weaknesses in the initial implementation.

In this step, we'll run our `PeopleExtractor` program on the test set and measure its accuracy using the evaluation framework defined earlier.

In [6]:
evaluate_correctness(extractor_program, test_set)

Average Metric: 45.00 / 50 (90.0%): 100%|██████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 662.61it/s]

2024/11/18 14:34:31 INFO dspy.evaluate.evaluate: Average Metric: 45 / 50 (90.0%)


,tokens,expected_extracted_people,prediction,extraction_correctness_metric
0,"['Takuya', 'Takagi', 'scored', 'the', 'winner', 'in', 'the', '88th...","[Takuya, Takagi, Hiroshige, Yanagimoto, Salem, Bitar]","[Takuya, Takagi, Hiroshige, Yanagimoto, Salem, Bitar]",✔️ [True]
1,"[It, was, the, second, costly, blunder, by, Syria, in, four, minut...",[],[],✔️ [True]
2,"['Defender', 'Hassan', 'Abbas', 'rose', 'to', 'intercept', 'a', 'l...","[Hassan, Abbas, Bitar]","[Hassan, Abbas, Bitar]",✔️ [True]
3,"[Nader, Jokhadar, had, given, Syria, the, lead, with, a, well-stru...","[Nader, Jokhadar]","[Nader, Jokhadar]",✔️ [True]
4,"[Japan, then, laid, siege, to, the, Syrian, penalty, area, for, mo...",[],[],✔️ [True]
5,"[Bitar, pulled, off, fine, saves, whenever, they, did, .]",[Bitar],[Bitar],✔️ [True]
6,"[Japan, coach, Shu, Kamo, said, :, ', ', The, Syrian, own, goal, p...","[Shu, Kamo]","[Shu, Kamo]",✔️ [True]
7,"[The, Syrians, scored, early, and, then, played, defensively, and,...",[],[],✔️ [True]
8,['],[],[],✔️ [True]
9,"['Japan', ',', 'co-hosts', 'of', 'the', 'World', 'Cup', 'in', '200...",[],[],✔️ [True]


## Optimize the Model

DSPy includes powerful optimizers that can improve the quality of your system.

Here, we use DSPy's `BootstrapFewShotWithRandomSearch` optimizer to:
- Automatically tune the program's language model (LM) prompt by incorporating helpful few-shot examples from the training dataset.
- Maximize correctness on the training set without overfitting.

This optimization process is automated, saving time and effort while improving accuracy.

The following optimization cell may run for several minutes.

In [12]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

optimizer = BootstrapFewShotWithRandomSearch(
    metric=extraction_correctness_metric,
    max_bootstrapped_demos=4,
    num_threads=24
)
optimized_extractor_program = optimizer.compile(extractor_program, trainset=train_set)

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 16 candidate sets.
Average Metric: 46.00 / 50 (92.0%): 100%|█████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 2565.26it/s]

2024/11/18 14:37:53 INFO dspy.evaluate.evaluate: Average Metric: 46 / 50 (92.0%)



New best score: 92.0 for seed -3
Scores so far: [92.0]
Best score so far: 92.0
Average Metric: 46.00 / 50 (92.0%): 100%|█████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 1804.95it/s]

2024/11/18 14:37:53 INFO dspy.evaluate.evaluate: Average Metric: 46 / 50 (92.0%)



Scores so far: [92.0, 92.0]
Best score so far: 92.0


...

  4%|████                                                                                                  | 2/50 [00:00<00:00, 1546.86it/s]

Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.


Average Metric: 37.00 / 50 (74.0%): 100%|██████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 765.73it/s]


2024/11/18 14:37:54 INFO dspy.evaluate.evaluate: Average Metric: 37 / 50 (74.0%)


Scores so far: [92.0, 92.0, 94.0, 92.0, 78.0, 88.0, 76.0, 88.0, 84.0, 80.0, 94.0, 98.0, 78.0, 98.0, 98.0, 74.0, 98.0, 84.0, 74.0]
Best score so far: 98.0
19 candidate programs found.


## Inspect Optimized Program's Prompt

After optimizing the program, we can inspect the history of interactions to see how DSPy has augmented the program's prompt with few-shot examples. This step demonstrates:
- The structure of the prompt used by the program.
- How few-shot examples are added to guide the model's behavior.

Use `inspect_history(n=1)` to view the last interaction and analyze the generated prompt.

In [8]:
dspy.inspect_history(n=1)





[2024-11-18T14:37:19.194494]

System message:

Your input fields are:
1. `tokens` (list[str]): Tokenized text, which may or may not contain tokens referring to people

Your output fields are:
1. `extracted_people` (list[str]): A list of all tokens referring to specific people extracted from the tokenized text
2. `rationale` (str): A detailed rationale for why each token referring to a person / people was extracted and why no additional tokens were extracted

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## tokens ## ]]
{tokens}

[[ ## extracted_people ## ]]
{extracted_people}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## rationale ## ]]
{rationale}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Extract tokens referring to specific people from a list of string tokens.
        
        The to

## Evaluate Optimized Program

After optimization, we re-evaluate the program on the test set to measure improvements. Comparing the optimized and initial results allows us to:
- Quantify the benefits of optimization.
- Validate that the program generalizes well to unseen data.

In this case, we see that accuracy of the program on the test dataset has improved.

In [9]:
evaluate_correctness(optimized_extractor_program, test_set)

Average Metric: 49.00 / 50 (98.0%): 100%|███████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.44it/s]

2024/11/18 14:37:28 INFO dspy.evaluate.evaluate: Average Metric: 49 / 50 (98.0%)


,tokens,expected_extracted_people,prediction,extraction_correctness_metric
0,"['Takuya', 'Takagi', 'scored', 'the', 'winner', 'in', 'the', '88th...","[Takuya, Takagi, Hiroshige, Yanagimoto, Salem, Bitar]","[Takuya, Takagi, Hiroshige, Yanagimoto, Salem, Bitar]",✔️ [True]
1,"[It, was, the, second, costly, blunder, by, Syria, in, four, minut...",[],[],✔️ [True]
2,"['Defender', 'Hassan', 'Abbas', 'rose', 'to', 'intercept', 'a', 'l...","[Hassan, Abbas, Bitar]","[Hassan, Abbas, Bitar]",✔️ [True]
3,"[Nader, Jokhadar, had, given, Syria, the, lead, with, a, well-stru...","[Nader, Jokhadar]","[Nader, Jokhadar]",✔️ [True]
4,"[Japan, then, laid, siege, to, the, Syrian, penalty, area, for, mo...",[],[],✔️ [True]
5,"[Bitar, pulled, off, fine, saves, whenever, they, did, .]",[Bitar],[Bitar],✔️ [True]
6,"[Japan, coach, Shu, Kamo, said, :, ', ', The, Syrian, own, goal, p...","[Shu, Kamo]","[Shu, Kamo]",✔️ [True]
7,"[The, Syrians, scored, early, and, then, played, defensively, and,...",[],[],✔️ [True]
8,['],[],[],✔️ [True]
9,"['Japan', ',', 'co-hosts', 'of', 'the', 'World', 'Cup', 'in', '200...",[],[],✔️ [True]


## Keeping an eye on cost

DSPy allows you to track the cost of your programs. The following code demonstrates how to obtain the cost of all LM calls made by the DSPy extractor program so far.

In [10]:
cost = sum([x['cost'] for x in lm.history if x['cost'] is not None])  # cost in USD, as calculated by LiteLLM for certain providers
cost

0.21780726000000009

## Saving and Loading Optimized Programs

DSPy supports saving and loading programs, enabling you to reuse optimized systems without the need to re-optimize from scratch. This feature is especially useful for deploying your programs in production environments or sharing them with collaborators.

In this step, we'll save the optimized program to a file and demonstrate how to load it back for future use.

In [11]:
optimized_extractor_program.save("optimized_extractor.json")

loaded_extractor_program = PeopleExtractor()
loaded_extractor_program.load("optimized_extractor.json")

loaded_extractor_program(tokens=["Italy", "recalled", "Marcello", "Cuttitta"])

['Marcello', 'Cuttitta']

## Conclusion

In this tutorial, we demonstrated how to:
- Use DSPy to build a modular, interpretable system for entity extraction.
- Evaluate and optimize the system using DSPy's built-in tools.

By leveraging structured inputs and outputs, we ensured that the system was easy to understand and improve. The optimization process allowed us to quickly improve performance without manually crafting prompts or tweaking parameters.

**Next Steps:**
- Experiment with extraction of other entity types (e.g., locations or organizations).
- Explore DSPy's other builtin modules like `ChainOfThought` for more complex reasoning tasks.
- Use the system in larger workflows, such as large scale document processing or summarization.